In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Folseek cluster meta and stats files

In [2]:
file = "./mmseqsCluster50-90_cluster.tsv"
col_names = ["cluster_representative", "cluster_member"]
df = pd.read_csv(file, sep="\t", names=col_names)

In [3]:
df

,cluster_representative,cluster_member
0,AAL57904.1.10_10815,AAL57904.1.10_10815
1,AAL57904.1.10_10815,ACG63545.1.10_10816
2,AIK23440.1.1_10913,AIK23440.1.1_10913
3,AIA62264.1.1.8_10168,AIA62264.1.1.8_10168
4,AAK91591.1.10_10416,AAK91591.1.10_10416
...,...,...
85157,ARD71343.1_53,ARD71343.1_53
85158,AOC55228.1_11899,AOC55228.1_11899
85159,AOC55228.1_11899,QHR78498.1_11900
85160,AOC55228.1_11899,AAU11039.1_11898


In [4]:
meta_file = "~/2_protein_structure_prediction/virosphere-fold-v1_predicted_dataset.csv"
meta_df = pd.read_csv(meta_file)

In [5]:
meta_df

,record_id,uniprot_id,pept_cat,protlen,genbank_name,gene,product,note,protein_id,mat_pept_id,...,protein_coordinates,esmfold_log_pLDDT,esmfold_log_pTM,colabfold_json_pLDDT,colabfold_json_pTM,PC1,PC2,PC3,protein_seq,structure_seq
0,CAX33877.1.4_11504,D3GMG2,mat_pept,11,Pep11 peptide,NaN,polyprotein,pep11 peptide,CAX33877.1,CAX33877.1.4,...,[518:529],69.9,0.016,84.4,0.04,2.951842,105.300208,99.588007,ASGTPITRASA,ASGTPITRASA
1,AAT67220.1.6_10459,Q672R3,mat_pept,12,2A,NaN,polyprotein,2A,AAT67220.1,AAT67220.1.6,...,[1295:1307],73.9,0.016,74.0,0.04,-600.036550,-364.558726,-107.356798,VFAHKQGPVTFQ,VFAHKQGPVTFQ
2,AAA73159.1_10363,Q88580,protein,12,Product: hypothetical protein,NaN,hypothetical protein,NaN,AAA73159.1,NaN,...,NaN,59.1,0.015,72.7,0.04,-561.185429,-315.068801,-88.217040,MACKHGYPDVCP,MACKHGYPDVCP
3,CAD30689.1.5_11499,Q8AZM0,mat_pept,12,VP2d protein,NaN,polyprotein,VP2d protein,CAD30689.1,CAD30689.1.5,...,[474:486],65.6,0.014,82.2,0.04,-71.142432,98.315052,61.274730,ASGTFSKRIPLA,ASGTFSKRIPLA
4,AGZ63355.1.1.1_10906,U5XL86,region,12,Polypyrimidine tracts,NaN,polyprotein,polypyrimidine tracts,AGZ63355.1,NaN,...,[1:13],69.8,0.016,80.0,0.04,-555.625112,-316.217022,-97.881186,PFSFSTPSPFPP,PFSFSTPSPFPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85157,ALZ45814.1_12442,A0A0X9NF79,protein,4182,Product: BUD22,NaN,BUD22,NaN,ALZ45814.1,NaN,...,NaN,NaN,NaN,27.3,0.41,340.703574,28.478686,-299.776917,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...
85158,ALN66244.1_12439,A0A0S2E616,protein,4183,Product: hypothetical protein; Note: wsv343,NaN,hypothetical protein,wsv343,ALN66244.1,NaN,...,NaN,NaN,NaN,26.8,0.40,332.364112,27.672160,-294.827197,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...
85159,ASA40472.1_12441,A0A1Z2R9K7,protein,4184,Product: wsv343,NaN,wsv343,NaN,ASA40472.1,NaN,...,NaN,NaN,NaN,27.5,0.40,343.191074,31.877971,-289.563303,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...
85160,ATU83352.1_12444,A0A2D3I4N7,protein,4184,Product: ORF963,NaN,ORF963,analog to wsv343; hypothetical protein,ATU83352.1,NaN,...,NaN,NaN,NaN,27.8,0.40,341.801376,34.613972,-292.724484,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...,MFKANVLNLGGGKFLESDVRDHLIKCANQMKEEPTTLRICLSNKLP...


In [6]:
# Add class of the representative structure to the meta_df
meta_df['record_id_class'] = meta_df.apply(
    lambda row: 'EF' if row['esmfold_log_pLDDT'] > 50 and row['esmfold_log_pLDDT'] > row['colabfold_json_pLDDT'] else 'CF', 
    axis=1)

In [7]:
# merge df and meta_df[['record_id', 'record_id_class']]
df = df.merge(meta_df[['record_id', 'record_id_class']], left_on="cluster_member", right_on="record_id")
df.drop(columns=["record_id"], inplace=True)

In [8]:
# rename record_id_class to member_class
df.rename(columns={"record_id_class": "member_class"}, inplace=True)
df['member_record_id'] = df['cluster_member']

In [9]:
# for each cluster_member in df, get corresponding protlen, plddd and ptm from meta_df
df['protlen'] = df['member_record_id'].map(meta_df.set_index('record_id')['protlen'])
df['ictv_species'] = df['member_record_id'].map(meta_df.set_index('record_id')['Species'])
#df['taxid'] = df['member_record_id'].map(meta_df.set_index('record_id')['taxid'])
#df['species'] = df['member_record_id'].map(meta_df.set_index('record_id')['Species'])

In [10]:
# for each cluster_member in df, get colabfold_json_pLDDT from meta_df if it has member_class == 'CF'
df['cf_plddt'] = df[df['member_class'] == 'CF']['member_record_id'].map(meta_df.set_index('record_id')['colabfold_json_pLDDT'])
df['cf_ptm'] = df[df['member_class'] == 'CF']['member_record_id'].map(meta_df.set_index('record_id')['colabfold_json_pTM'])

In [11]:
# for each cluster_member in df, get esmfold_log_pLDDT from meta_df if it has member_class == 'EF'
df['ef_plddt'] = df[df['member_class'] == 'EF']['member_record_id'].map(meta_df.set_index('record_id')['esmfold_log_pLDDT'])
df['ef_ptm'] = df[df['member_class'] == 'EF']['member_record_id'].map(meta_df.set_index('record_id')['esmfold_log_pTM'])

In [12]:
# fuse columns of plddd and ptm
df['plddt'] = df['cf_plddt'].combine_first(df['ef_plddt'])
df['ptm'] = df['cf_ptm'].combine_first(df['ef_ptm'])

In [13]:
# drop columns cf_plddd, cf_ptm, ef_plddd, ef_ptm
df = df.drop(columns=['cf_plddt', 'cf_ptm', 'ef_plddt', 'ef_ptm'])

In [14]:
df

,cluster_representative,cluster_member,member_class,member_record_id,protlen,ictv_species,plddd,ptm
0,AAL57904.1.10_10815,AAL57904.1.10_10815,CF,AAL57904.1.10_10815,21,Sapelovirus B,73.1,0.080
1,AAL57904.1.10_10815,ACG63545.1.10_10816,CF,ACG63545.1.10_10816,22,Sapelovirus B,74.5,0.090
2,AIK23440.1.1_10913,AIK23440.1.1_10913,EF,AIK23440.1.1_10913,21,Sicinivirus A,60.1,0.045
3,AIA62264.1.1.8_10168,AIA62264.1.1.8_10168,CF,AIA62264.1.1.8_10168,21,Nyctalus velutinus alphacoronavirus SC-2013,86.6,0.160
4,AAK91591.1.10_10416,AAK91591.1.10_10416,CF,AAK91591.1.10_10416,21,Erbovirus A,77.7,0.130
...,...,...,...,...,...,...,...,...
85157,ARD71343.1_53,ARD71343.1_53,CF,ARD71343.1_53,248,Mardivirus columbidalpha1,68.2,0.530
85158,AOC55228.1_11899,AOC55228.1_11899,CF,AOC55228.1_11899,248,Lymphocystis disease virus 3,86.8,0.840
85159,AOC55228.1_11899,QHR78498.1_11900,CF,QHR78498.1_11900,249,Lymphocystis disease virus 4,87.9,0.860
85160,AOC55228.1_11899,AAU11039.1_11898,CF,AAU11039.1_11898,249,Lymphocystis disease virus 2,87.1,0.850


In [15]:
df.to_csv("./mmseqsCluster50-90_meta.csv", index=False)

In [16]:
df.isna().sum()

cluster_representative    0
cluster_member            0
member_class              0
member_record_id          0
protlen                   0
ictv_species              0
plddd                     0
ptm                       0
dtype: int64

In [17]:
cluster_reps = df["cluster_representative"].unique().tolist()

In [18]:
len(cluster_reps)

33151

In [19]:
# count the number of cluster_members for each cluster_representative, sort by count and make it a dataframe
cluster_counts = df["cluster_representative"].value_counts().reset_index()
cluster_counts.columns = ["cluster_representative", "count"]
cluster_counts

,cluster_representative,count
0,AMW87255.1.1.10_10632,513
1,AAX47041.1.1.1_10542,498
2,AHA35211.1.1.7_10792,364
3,AHJ81401.1_4666,317
4,AAQ73083.1.1.9_10517,304
...,...,...
33146,QED55973.1_11822,1
33147,AER41487.1_12398,1
33148,AAQ96431.1_12417,1
33149,CCU56449.1_12049,1


In [20]:
df_slice = df.drop(columns=['cluster_member', 'member_record_id', 'member_class', 'ictv_species'])

In [21]:
df_slice

,cluster_representative,protlen,plddd,ptm
0,AAL57904.1.10_10815,21,73.1,0.080
1,AAL57904.1.10_10815,22,74.5,0.090
2,AIK23440.1.1_10913,21,60.1,0.045
3,AIA62264.1.1.8_10168,21,86.6,0.160
4,AAK91591.1.10_10416,21,77.7,0.130
...,...,...,...,...
85157,ARD71343.1_53,248,68.2,0.530
85158,AOC55228.1_11899,248,86.8,0.840
85159,AOC55228.1_11899,249,87.9,0.860
85160,AOC55228.1_11899,249,87.1,0.850


In [22]:
# for each cluster_representative, get protlen_mean, plddt_mean, ptm_mean from df
cluster_means = df_slice.groupby("cluster_representative").mean().reset_index()
cluster_means

,cluster_representative,protlen,plddd,ptm
0,AAA02870.1_119,864.0,47.80,0.395
1,AAA03189.1.1_6577,182.5,89.75,0.815
2,AAA03189.1.2_6577,217.0,70.70,0.560
3,AAA03189.1.3_6577,160.0,70.00,0.440
4,AAA03189.1.4_6577,67.0,72.40,0.200
...,...,...,...,...
33146,YP_094050.1_12135,294.0,66.50,0.562
33147,YP_094051.1_12135,111.0,89.00,0.810
33148,YP_094052.1_12135,244.0,87.60,0.860
33149,YP_094053.1_12135,253.0,83.50,0.860


In [23]:
# fuse cluster_counts and cluster_means dataframes
cluster_stats = pd.merge(cluster_counts, cluster_means, on="cluster_representative")
cluster_stats

,cluster_representative,count,protlen,plddd,ptm
0,AMW87255.1.1.10_10632,513,455.957115,95.771540,0.937076
1,AAX47041.1.1.1_10542,498,68.283133,63.245783,0.243594
2,AHA35211.1.1.7_10792,364,98.799451,92.443956,0.828516
3,AHJ81401.1_4666,317,511.337539,84.447634,0.808139
4,AAQ73083.1.1.9_10517,304,172.960526,95.915789,0.913520
...,...,...,...,...,...
33146,QED55973.1_11822,1,184.000000,61.100000,0.250000
33147,AER41487.1_12398,1,184.000000,94.400000,0.890000
33148,AAQ96431.1_12417,1,184.000000,38.000000,0.310000
33149,CCU56449.1_12049,1,184.000000,76.700000,0.700000


In [24]:
cluster_stats

,cluster_representative,count,protlen,plddd,ptm
0,AMW87255.1.1.10_10632,513,455.957115,95.771540,0.937076
1,AAX47041.1.1.1_10542,498,68.283133,63.245783,0.243594
2,AHA35211.1.1.7_10792,364,98.799451,92.443956,0.828516
3,AHJ81401.1_4666,317,511.337539,84.447634,0.808139
4,AAQ73083.1.1.9_10517,304,172.960526,95.915789,0.913520
...,...,...,...,...,...
33146,QED55973.1_11822,1,184.000000,61.100000,0.250000
33147,AER41487.1_12398,1,184.000000,94.400000,0.890000
33148,AAQ96431.1_12417,1,184.000000,38.000000,0.310000
33149,CCU56449.1_12049,1,184.000000,76.700000,0.700000


In [25]:
cluster_stats["protlen_mean"] = cluster_stats["protlen"].round(2)
cluster_stats["plddt_mean"] = cluster_stats["plddd"].round(2)
cluster_stats["ptm_mean"] = cluster_stats["ptm"].round(2)
cluster_stats = cluster_stats.drop(columns=['protlen', 'plddd', 'ptm'])
cluster_stats.columns = ["cluster_representative", "cluster_size", "protlen_mean", "plddt_mean", "ptm_mean"]

In [26]:
cluster_stats.to_csv("./mmseqsCluster50-90_stats.csv", index=False)

In [27]:
cluster_stats_ns = cluster_stats[cluster_stats['cluster_size'] > 1]

In [28]:
cluster_stats_ns

,cluster_representative,cluster_size,protlen_mean,plddt_mean,ptm_mean
0,AMW87255.1.1.10_10632,513,455.96,95.77,0.94
1,AAX47041.1.1.1_10542,498,68.28,63.25,0.24
2,AHA35211.1.1.7_10792,364,98.80,92.44,0.83
3,AHJ81401.1_4666,317,511.34,84.45,0.81
4,AAQ73083.1.1.9_10517,304,172.96,95.92,0.91
...,...,...,...,...,...
10477,AJR28439.1_9032,2,510.00,75.45,0.45
10478,AIU39322.1_59,2,207.00,50.10,0.42
10479,ATI21234.1_12000,2,46.00,87.60,0.52
10480,ACU57013.1_12160,2,47.00,75.20,0.36


In [29]:
cluster_stats_ns[cluster_stats_ns['cluster_size'] >= 100]

,cluster_representative,cluster_size,protlen_mean,plddt_mean,ptm_mean
0,AMW87255.1.1.10_10632,513,455.96,95.77,0.94
1,AAX47041.1.1.1_10542,498,68.28,63.25,0.24
2,AHA35211.1.1.7_10792,364,98.80,92.44,0.83
3,AHJ81401.1_4666,317,511.34,84.45,0.81
4,AAQ73083.1.1.9_10517,304,172.96,95.92,0.91
5,ACK37410.1.1.2_10693,274,206.21,88.66,0.86
6,BAA24003.1.1.6_10589,243,99.11,71.24,0.51
7,CAA24465.1.2_10557,224,261.28,87.73,0.86
8,AEE01391.1.1.2_9626,180,743.81,86.35,0.84
9,ABO69379.1.1.5_10722,171,126.68,94.49,0.88
